# Zero to Singularity: Create, Tune, Deploy and Scale a Deep Neural Network in 90 Minutes

This notebook is part of a masterclass held at IBM Think on 13th of February 2019 in San Fransisco
In this exercise you will train a Keras DeepLearning model running on top of TensorFlow. 

Note: For sake of bringing the training runtime down we've done two things

1) Used a softmax regression model over a Convolutional Neural Network 

2) Trained only for one epoch instead of 20

This leads to approx. 5% less accuracy


Authors

Romeo Kienzler - Chief Data Scientist, IBM Watson IoT

Krishnamurthy Arthanarisamy - Architect, Watson Machine Learning Software Lab, Bangalore


# Prerequisites

Please make sure the currently installed version of Keras and Tensorflow are matching the requirememts, if not, please run the two PIP commands below in order to re-install. Please restart the kernal before proceeding, please re-check if the versions are matching.

In [1]:
import keras
print('Current:\t', keras.__version__)
print('Expected:\t 2.2.5 ')

Using TensorFlow backend.


Current:	 2.2.4
Expected:	 2.2.5 


In [2]:
import tensorflow as tf
print('Current:\t', tf.__version__)
print('Expected:\t 1.15.0')

Current:	 1.13.1
Expected:	 1.15.0


# IMPORTANT !!!

If you ran the two lines below please restart your kernel (Kernel->Restart & Clear Output)

In [3]:
!pip install keras==2.2.5 

     |████████████████████████████████| 337kB 9.5MB/s eta 0:00:01
     |████████████████████████████████| 51kB 22.5MB/s eta 0:00:01
     |████████████████████████████████| 51kB 12.9MB/s eta 0:00:01
ERROR: tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.
  Found existing installation: Keras-Preprocessing 1.0.5
    Uninstalling Keras-Preprocessing-1.0.5:
      Successfully uninstalled Keras-Preprocessing-1.0.5
  Found existing installation: Keras-Applications 1.0.6
    Uninstalling Keras-Applications-1.0.6:
      Successfully uninstalled Keras-Applications-1.0.6
  Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


In [4]:
!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 74kB/s s eta 0:00:01                      | 32.6MB 7.3MB/s eta 0:00:52          | 185.9MB 51.5MB/s eta 0:00:05     |█████████████████▉              | 229.3MB 41.3MB/s eta 0:00:05
     |████████████████████████████████| 61kB 19.5MB/s eta 0:00:01
     |████████████████████████████████| 14.6MB 28.9MB/s eta 0:00:01
     |████████████████████████████████| 512kB 42.9MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 39.4MB/s eta 0:00:01
     |████████████████████████████████| 71kB 27.0MB/s eta 0:00:01
     |████████████████████████████████| 798kB 39.8MB/s eta 0:00:01
ERROR: autoai-libs 1.10.5 has requirement pandas>=0.24.2, but you'll have pandas 0.24.1 which is incompatible.
  Found existing installation: numpy 1.15.4
    Uninstalling numpy-1.15.4:
      Successfully uninstalled numpy-1.15.4
  Found existing installation: tensorflow-estimator 1.13.0
    Uninstalling tensorflow-estimator-1.13.0:
      Successfully uninstalled tensor

# 1.0 Train a MNIST digits recognition model
We start with some global parameters and imports

In [5]:
#some learners constantly reported 502 errors in Watson Studio. 
#This is due to the limited resources in the free tier and the heavy resource consumption of Keras.
#This is a workaround to limit resource consumption

from keras import backend as K

K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))

In [6]:
import keras
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.optimizers import RMSprop
from keras.layers import LeakyReLU

from keras import backend as K
import numpy as np

In [8]:
batch_size = 128
num_classes = 10
epochs = 1

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


11493376/11490434 [==============================] - 0s 0us/step


# Training a simple model
First we'll train a simple softmax regressor and check what accuracy we get

In [9]:
model = Sequential()
model.add(Dense(512, input_shape=(784,)))
model.add(Dense(num_classes, activation='softmax'))



model.compile(loss='categorical_crossentropy',
        optimizer='rmsprop',
        metrics=['accuracy'])

model.fit(x_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        validation_data=(x_test, y_test))
        
score = model.evaluate(x_test, y_test, verbose=0)

print('\n')
print('Accuracy:',score[1])

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 8s 139us/step - loss: 0.3838 - acc: 0.8892 - val_loss: 0.3299 - val_acc: 0.9040


Accuracy: 0.904


In [10]:
#some cleanup from the previous run
!rm -f ker_*
!rm -f my_best_model.tgz

You should see an accuracy of approximately 90%. Now lets define a hyper-parameter grid including different activation functions and gradient descent optimizers. We’re optimizing over the grid using grid search (nested for loops) and store each model variant in a file. We then decide for the best one in order to deploy to IBM Watson Machine Learning.

In [11]:
#define parameter grid

activation_functions_layer_1 = ['sigmoid','tanh','relu']
opimizers = ['rmsprop','adagrad','adadelta']

#optimize over parameter grid (grid search)

for activation_function_layer_1 in activation_functions_layer_1:
    for opimizer in opimizers:
        
        model = Sequential()
        model.add(Dense(512, activation = activation_function_layer_1, input_shape=(784,)))
        model.add(Dense(num_classes, activation='softmax'))



        model.compile(loss='categorical_crossentropy',
              optimizer=opimizer,
              metrics=['accuracy'])

        model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
        
        score = model.evaluate(x_test, y_test, verbose=0)
        save_path = "ker_func_mnist_model_2.%s.%s.%s.h5" % (activation_function_layer_1,opimizer,score[1])
        model.save(save_path)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 10s 163us/step - loss: 0.4206 - acc: 0.8872 - val_loss: 0.2579 - val_acc: 0.9263
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 10s 162us/step - loss: 0.4101 - acc: 0.8900 - val_loss: 0.2886 - val_acc: 0.9179
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 11s 176us/step - loss: 0.5476 - acc: 0.8610 - val_loss: 0.3338 - val_acc: 0.9025
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 7s 114us/step - loss: 0.3314 - acc: 0.9025 - val_loss: 0.1963 - val_acc: 0.9433
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 9s 151us/step - loss: 0.3239 - acc: 0.9082 - val_loss: 0.2325 - val_acc: 0.9333
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60

# Model evaluation
Let's have a look at all the models and see which hyper parameter configuration was the best one. You should see that relu and rmsprop gives you > 95% of accuracy on the validation set

In [12]:
ls -ltr ker_*

-rw-r----- 1 dsxuser dsxuser 3276560 Jul  9 16:47 ker_func_mnist_model_2.sigmoid.rmsprop.0.9263.h5
-rw-r----- 1 dsxuser dsxuser 3276560 Jul  9 16:47 ker_func_mnist_model_2.sigmoid.adagrad.0.9179.h5
-rw-r----- 1 dsxuser dsxuser 4905120 Jul  9 16:48 ker_func_mnist_model_2.sigmoid.adadelta.0.9025.h5
-rw-r----- 1 dsxuser dsxuser 3276568 Jul  9 16:48 ker_func_mnist_model_2.tanh.rmsprop.0.9433.h5
-rw-r----- 1 dsxuser dsxuser 3276568 Jul  9 16:48 ker_func_mnist_model_2.tanh.adagrad.0.9333.h5
-rw-r----- 1 dsxuser dsxuser 4905128 Jul  9 16:48 ker_func_mnist_model_2.tanh.adadelta.0.9322.h5
-rw-r----- 1 dsxuser dsxuser 3276568 Jul  9 16:48 ker_func_mnist_model_2.relu.rmsprop.0.9561.h5
-rw-r----- 1 dsxuser dsxuser 3276568 Jul  9 16:49 ker_func_mnist_model_2.relu.adagrad.0.9605.h5
-rw-r----- 1 dsxuser dsxuser 4905128 Jul  9 16:49 ker_func_mnist_model_2.relu.adadelta.0.9496.h5


Now it's time to create a tarball out of your favorite model, please replace the name of your favorite model H5 file with “please-put-me-here”

In [14]:
!tar -zcvf my_best_model.tgz ker_func_mnist_model_2.relu.adagrad.0.9605.h5

ker_func_mnist_model_2.relu.adagrad.0.9605.h5


## 2.0 Save the trained model to WML Repository

We will use `watson_machine_learning_client` python library to save the trained model to WML Repository, to deploy the saved model and to make predictions using the deployed model.</br>


`watson_machine_learning_client` can be installed using the following `pip` command in case you are running outside Watson Studio:

`!pip install watson-machine-learning-client --upgrade`

In [15]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

2020-07-09 16:50:36,227 - watson_machine_learning_client.wml_client_error - WARNING - Error during getting IAM Token. (POST https://iam.cloud.ibm.com/oidc/token)
Status code: 400, body: {"context":{"requestId":"3e6711fffe3e4520a8c4737e5d94d427","requestType":"incoming.OIDC_Token","userAgent":"python-requests/2.21.0","url":"https://iam.cloud.ibm.com","instanceId":"iamid-5.7-8148-aa50811-7f4556df6c-bcmw2","threadId":"d035d","host":"iamid-5.7-8148-aa50811-7f4556df6c-bcmw2","startTime":"09.07.2020 16:50:36:128 GMT","endTime":"01.01.1970 00:00:00:000 GMT","elapsedTime":"0","locale":"en_US","clusterName":"iam-id-prdal12-8brn"},"errorCode":"BXNIM0415E","errorMessage":"Provided API key could not be found"}
2020-07-09 17:04:20,424 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.
2020-07-09 17:05:10,197 - watson_machine_learning_client.wml_client_error - WARNING - 'deployment_uid' is not an uid: 'PASTE_YOUR_GUID_HERE_IF_APPLICABLE

Please go to https://cloud.ibm.com/,  login, click on the “Create Resource” button. From the “AI” category, please choose “Machine Learning”. Wait for the “Create” button to activate and click on “Create”. Click on “Service Credentials”, then “New Credential”, then “Add”. From the new entry in the table, under “ACTIONS”, please click on “View Credentials”. Please copy the whole JSON object to your clipboard. Now just paste the JSON object below so that you are able to use your personal instance of Watson Machine Learning.

In [18]:
wml_credentials={
  "apikey": "2aj6uq66x_tqgEfvq1FGOZ8PwD4X-Cqe2XjaIriyLD75",
  "iam_apikey_description": "Auto-generated for key 13cff054-730f-4330-9c91-4496004dae75",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/ab651b4ec4824a3c91c7b79e4e3b4d37::serviceid:ServiceId-efddd192-9712-4de2-8a90-254e61d0c2c4",
  "instance_id": "647889d3-fd3b-4f2f-b647-c444cb9e850e",
  "url": "https://us-south.ml.cloud.ibm.com"
}

In [19]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [20]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "IBM", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "ibm@ibm.com", 
               client.repository.ModelMetaNames.NAME: "KK3_clt_keras_mnist",
               client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow",
               client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.15" ,
               client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES: [{"name": "keras", "version": "2.2.5"}]
              }

In [21]:
published_model = client.repository.store_model(model="my_best_model.tgz", meta_props=model_props)


In [22]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)

## 3.0 Deploy the Keras model

In [23]:
client.deployments.list()

----  ----  ----  -----  -------  ---------  -------------
GUID  NAME  TYPE  STATE  CREATED  FRAMEWORK  ARTIFACT TYPE
----  ----  ----  -----  -------  ---------  -------------


To keep your environment clean, just delete all deployments from previous runs

In [24]:
client.deployments.delete("PASTE_YOUR_GUID_HERE_IF_APPLICABLE")

WMLClientError: 'deployment_uid' is not an uid: 'PASTE_YOUR_GUID_HERE_IF_APPLICABLE'

In [25]:
created_deployment = client.deployments.create(published_model_uid, name="k1_keras_mnist_clt1")



#######################################################################################

Synchronous deployment creation for uid: '9fa54e11-64fd-47a4-ba31-f6461d66aa46' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS.
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='85d336d4-7f91-4883-8f79-28fa78709c25'
------------------------------------------------------------------------------------------------




## Test the model

In [26]:
#scoring_endpoint = client.deployments.get_scoring_url(created_deployment)
scoring_endpoint = created_deployment['entity']['scoring_url']
print(scoring_endpoint)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/647889d3-fd3b-4f2f-b647-c444cb9e850e/deployments/85d336d4-7f91-4883-8f79-28fa78709c25/online


In [27]:
x_score_1 = x_test[23].tolist()
print('The answer should be: ',np.argmax(y_test[23]))
scoring_payload = {'values': [x_score_1]}

The answer should be:  5


In [28]:
predictions = client.deployments.score(scoring_endpoint, scoring_payload)
print('And the answer is!... ',predictions['values'][0][1])

And the answer is!...  5
